In [75]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from scipy import stats

In [76]:
df_saeb_rend = pd.read_csv('data/saeb_rend.csv', delimiter=',')

df_saeb_rend['codigo'] = df_saeb_rend['codigo'].astype('str')

# Estimando o modelo
modelo_saeb = sm.OLS.from_formula('saeb ~ rendimento', df_saeb_rend).fit()

# Parâmetros do 'modelo_saeb'
print(modelo_saeb.summary())

                            OLS Regression Results                            
Dep. Variable:                   saeb   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     2126.
Date:                Fri, 30 Aug 2024   Prob (F-statistic):               0.00
Time:                        22:46:39   Log-Likelihood:                -27984.
No. Observations:               25530   AIC:                         5.597e+04
Df Residuals:                   25528   BIC:                         5.599e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2425      0.039     82.277      0.0

In [77]:
df_saeb_rend['fitted'] = modelo_saeb.fittedvalues
df_saeb_rend['residuos'] = modelo_saeb.resid

In [78]:
# Teste de Breusch-Pagan (Cálculo Manual) - REZA!

df_saeb_rend['up'] = ((df_saeb_rend['residuos'])**2)/\
    (((df_saeb_rend['residuos'])**2).sum()/25530)
    
modelo_aux = sm.OLS.from_formula('up ~ fitted',
                                 df_saeb_rend).fit()

print(modelo_aux.summary())

                            OLS Regression Results                            
Dep. Variable:                     up   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     13.97
Date:                Fri, 30 Aug 2024   Prob (F-statistic):           0.000186
Time:                        22:46:39   Log-Likelihood:                -47370.
No. Observations:               25530   AIC:                         9.474e+04
Df Residuals:                   25528   BIC:                         9.476e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1258      0.234      0.537      0.5

In [79]:
anova_table = anova_lm(modelo_aux, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
fitted,33.441401,1.0,13.966877,0.000186
Residual,61122.616057,25528.0,NaN,NaN


In [80]:
SQReg = anova_table.sum_sq.iloc[0]/2
SQReg

16.720700325953544

In [81]:
p_value = stats.chi2.pdf(SQReg, 1)*2
p_value

4.5651903149382985e-05

p-value < 0.05 portanto 'Rejeita-se H0 - Existência de Heterocedasticidade'

In [82]:
# Podemos evitar esse cálculo manual com a função:

# Criação da função 'breusch_pagan_test'
def breusch_pagan_test(modelo):

    df = pd.DataFrame({'yhat':modelo.fittedvalues,
                       'resid':modelo.resid})
  
    df['up'] = (np.square(df.resid))/np.sum(((np.square(df.resid))/df.shape[0]))
  
    modelo_aux = sm.OLS.from_formula('up ~ yhat', df).fit()
  
    anova_table = sm.stats.anova_lm(modelo_aux, typ=2)
  
    anova_table['sum_sq'] = anova_table['sum_sq']/2
    
    chisq = anova_table['sum_sq'].iloc[0]

    p_value = stats.chi2.pdf(chisq, 1)*2
    
    print(f"chisq: {chisq}")
    
    print(f"p-value: {p_value}")
    
    return chisq, p_value

In [83]:
# Teste de Breusch-Pagan com o modelo com dummies

teste_bp = breusch_pagan_test(modelo_saeb) #criação do objeto 'teste_bp'
chisq, p = teste_bp #definição dos elementos contidos no objeto 'teste_bp'
alpha = 0.05 #nível de significância
if p > alpha:
    print('Não se rejeita H0 - Ausência de Heterocedasticidade')
else:
	print('Rejeita-se H0 - Existência de Heterocedasticidade')

chisq: 16.720700325953544
p-value: 4.5651903149382985e-05
Rejeita-se H0 - Existência de Heterocedasticidade


Agora rodando a função para o modelo com dummies:

In [84]:
# Dummização da variável 'uf' com n-1 dummies

df_saeb_rend_dummies = pd.get_dummies(df_saeb_rend, columns=['uf'],
                                      dtype=int,
                                      drop_first=True)

df_saeb_rend_dummies.head(3) # n-1

,municipio,codigo,escola,rede,saeb,rendimento,fitted,residuos,up,uf_AL,...,uf_PR,uf_RJ,uf_RN,uf_RO,uf_RR,uf_RS,uf_SC,uf_SE,uf_SP,uf_TO
0,Alta Floresta D'Oeste,11024666,EMEIEF BOA ESPERANCA,Municipal,5.331833,0.766092,4.825556,0.506278,0.488848,0,...,0,0,0,1,0,0,0,0,0,0
1,Alta Floresta D'Oeste,11024682,EEEF EURIDICE LOPES PEDROSO,Estadual,NaN,0.910890,NaN,NaN,NaN,0,...,0,0,0,1,0,0,0,0,0,0
2,Alta Floresta D'Oeste,11024828,EMEIEF IZIDORO STEDILE,Municipal,5.432333,0.884658,5.070567,0.361767,0.249605,0,...,0,0,0,1,0,0,0,0,0,0


In [85]:
# Definição da fórmula utilizada no modelo
lista_colunas = list(df_saeb_rend_dummies.drop(columns=['municipio',
                                                        'codigo',
                                                        'escola',
                                                        'rede',
                                                        'saeb',
                                                        'fitted',
                                                        'residuos',
                                                        'up']).columns)
formula_dummies_modelo = ' + '.join(lista_colunas)
formula_dummies_modelo = "saeb ~ " + formula_dummies_modelo

# Estimação
modelo_saeb_dummies_uf = sm.OLS.from_formula(formula_dummies_modelo,
                                               df_saeb_rend_dummies).fit()

# Parâmetros do modelo 'modelo_saeb_dummies_uf'
print(modelo_saeb_dummies_uf.summary())

                            OLS Regression Results                            
Dep. Variable:                   saeb   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     497.5
Date:                Fri, 30 Aug 2024   Prob (F-statistic):               0.00
Time:                        22:46:40   Log-Likelihood:                -23604.
No. Observations:               25530   AIC:                         4.726e+04
Df Residuals:                   25502   BIC:                         4.749e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7566      0.071     53.118      0.0

In [86]:
# Teste de Breusch-Pagan com o modelo com dummies

teste_bp = breusch_pagan_test(modelo_saeb_dummies_uf) #criação do objeto 'teste_bp'
chisq, p = teste_bp #definição dos elementos contidos no objeto 'teste_bp'
alpha = 0.05 #nível de significância
if p > alpha:
    print('Não se rejeita H0 - Ausência de Heterocedasticidade')
else:
	print('Rejeita-se H0 - Existência de Heterocedasticidade')

chisq: 1.0756241898384158
p-value: 0.44930469428880987
Não se rejeita H0 - Ausência de Heterocedasticidade


In [87]:
# Dummização da variável 'uf' com n-1 dummies sendo uf_BA como referencia

df_saeb_rend_dummies = pd.get_dummies(df_saeb_rend, columns=['uf'],
                                      dtype=int,
                                      drop_first=False)

#df_saeb_rend_dummies

lista_colunas = list(df_saeb_rend_dummies.drop(columns=['municipio',
                                                        'codigo',
                                                        'escola',
                                                        'rede',
                                                        'saeb',
                                                        'fitted',
                                                        'residuos','up',
                                                        'uf_BA']).columns)
formula_dummies_modelo = ' + '.join(lista_colunas)
formula_dummies_modelo = "saeb ~ " + formula_dummies_modelo

# Estimação do novo modelo 'modelo_saeb_dummies_uf2'

modelo_saeb_dummies_uf2 = sm.OLS.from_formula(formula_dummies_modelo,
                                               df_saeb_rend_dummies).fit()

# Parâmetros do modelo 'modelo_saeb_dummies_uf2'
print(modelo_saeb_dummies_uf2.summary())

                            OLS Regression Results                            
Dep. Variable:                   saeb   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     497.5
Date:                Fri, 30 Aug 2024   Prob (F-statistic):               0.00
Time:                        22:46:40   Log-Likelihood:                -23604.
No. Observations:               25530   AIC:                         4.726e+04
Df Residuals:                   25502   BIC:                         4.749e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.5114      0.038     93.345      0.0

In [88]:
# Teste de Breusch-Pagan no novo modelo

teste_bp = breusch_pagan_test(modelo_saeb_dummies_uf2) #criação do objeto 'teste_bp'
chisq, p = teste_bp #definição dos elementos contidos no objeto 'teste_bp'
alpha = 0.05 #nível de significância
if p > alpha:
    print('Não se rejeita H0 - Ausência de Heterocedasticidade')
else:
	print('Rejeita-se H0 - Existência de Heterocedasticidade')

chisq: 1.0756241898380683
p-value: 0.4493046942889605
Não se rejeita H0 - Ausência de Heterocedasticidade


Provamos que não há diferença em mudar a variável de referência da dummiezação